Домашнее задание 8. Задача 1. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId

In [2]:
import pandas as pd

In [3]:
ratings = pd.read_csv('ratings8.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
over100_ratings = ratings.groupby('userId').count()
over100_ratings = over100_ratings[over100_ratings['rating']>=100]
over100_ratings = over100_ratings.merge(ratings, how = 'left', on = 'userId')
over100_ratings = over100_ratings.drop(['movieId_x', 'rating_x','timestamp_x'], axis=1)
maxtime = over100_ratings.groupby('userId')['timestamp_y'].max()
mintime = over100_ratings.groupby('userId')['timestamp_y'].min()
lifetime = (maxtime - mintime)/3600
lifetime.mean()

10925.485712927757

In [5]:
over100_ratings = ratings.groupby('userId').count()
over100_ratings = over100_ratings[over100_ratings['rating']>=100].reset_index()
over100_ratings_ids = over100_ratings['userId'].tolist()
over100_ratings_ids 
filt_ratings = ratings[ ratings['userId'].isin(over100_ratings_ids)]
maxtime1 = filt_ratings.groupby('userId')['timestamp'].max()
mintime1 = filt_ratings.groupby('userId')['timestamp'].min()
lifetime1 = (maxtime1 - mintime1)/3600
lifetime1.mean()

10925.485712927757

Домашнее задание 8. Задача 2.
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.
По заданию 2 - проверьте результат, который у вас получается после объединения таблиц. Он должен содержать все данные из исходных таблиц
### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [6]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [12]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [11]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [13]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [14]:
rail_merg = rzd.merge(client_base, how = 'right')
merg2 = rail_merg.merge(auto, how = 'left')
merg3 = merg2.merge(air,how = 'left')
final_merg = merg3.drop(merg3.columns[[2]], axis='columns')
final_merg

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [15]:
final_merg1 = rzd.merge(client_base, how = 'outer').merge(auto, how = 'left').merge(air,how = 'left')
right_order = [0,2,1,3,4]
final_merg1[final_merg1.columns[right_order]]

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


Домашнее задание 8. Задача 3.

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок?

В задании 3 дайте комментарии в свободной форме. Никакого точного ответа у задачи нет (при такой постановке)

    Поскольку нам дана последовательность координат, логично выделить из низ два-три наиболее встечающихся(хорошо, если каждой паре координат соответствует столбец времени фиксации местоположения, тогда можно определить конкретно где есть что, более того, по уникальному значению времени можно сформировать составной primary key, оп которому можно идентифицировать уникальную покупку, а отсюда, клиента и координаты , и джойнить таблицы), - и предположим, что таковые являются домом-работой-дачей(у нас даже категория одноимённая имеется). 
    Также признаком постоянного нахождения человека в конкретном месте могут служить близкие по расположению координаты. 
    Отдельный вопрос уже у меня - а как в python можно найти из наиболее повторяющееся из близких значений, скажем, в массиве json(допустим, клиент 11 гулял весь день возле офиса и оставил следы в виде геоданных вокруг ст.м.Белорусская) 
    Большое значение имеет и само предназначение приложения, если для его полноценной работы требуются геоданные. Здесь может играть роль и ситуация, когда приложение не имеет геоданных для полноценной работы, например, когда они у клиента отключены.  
    Итак, создадим д-фрейм координат и времени, в которое они были вычислены. Объединиим такой д-фрейм по столбцам координат и user_id  с таблицей из лекции.  

In [41]:
coords =  pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 77, 99],
        'latitude': [59.572794666070,
                     59.339912356602,
                     59.815321889482,
                     59.836000591413,
                     60.017146443298,
                     60.003150884291],
        'longitude': [30.198665570800,
                      30.193396405761,
                      30.231699902345,
                      30.773373212402,
                      30.223551708984,
                      30.645564069335],
        'timestamp': [1260759144,
                      1260759179,
                      1260759182,
                      1260759185,
                      1260758816,
                      1260759205]
    }
)
coords

,user_id,latitude,longitude,timestamp
0,11,59.572795,30.198666,1260759144
1,22,59.339912,30.193396,1260759179
2,55,59.815322,30.231700,1260759182
3,11,59.836001,30.773373,1260759185
4,77,60.017146,30.223552,1260758816
5,99,60.003151,30.645564,1260759205


In [49]:
visits = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 77],
        'source': ['ad', 'yandex', 'email', 'google', 'ad']
    }
)

visits = visits[['user_id', 'source']]
visits

,user_id,source
0,11,ad
1,22,yandex
2,55,email
3,11,google
4,77,ad


In [48]:
purchases = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'category': ['Спорт', 'Авто', 'Дача', 'Спорт', 'Авто']
    }
)

purchases = purchases[['user_id', 'category']]
purchases

,user_id,category
0,11,Спорт
1,22,Авто
2,55,Дача
3,11,Спорт
4,99,Авто


In [20]:
visits_grouped = visits.groupby('user_id').count()
visits_grouped.rename(columns={'source': 'visits'}, inplace=True)
visits_grouped

,visits
user_id,
11,2
22,1
55,1
77,1


In [50]:
purchases_pivot = purchases.pivot_table(index='user_id', columns='category', values='user_id', 
                                        aggfunc='size', fill_value=0)
purchases_pivot

category,Авто,Дача,Спорт
user_id,,,
11,0,0,2
22,1,0,0
55,0,1,0
99,1,0,0


In [45]:
merged = visits_grouped.join(purchases_pivot, how='outer').reset_index()
merged

,user_id,visits,Авто,Дача,Спорт
0,11,2.0,0.0,0.0,2.0
1,22,1.0,1.0,0.0,0.0
2,55,1.0,0.0,1.0,0.0
3,77,1.0,NaN,NaN,NaN
4,99,NaN,1.0,0.0,0.0


In [47]:
coords.merge(merged,how = 'left', on=["user_id"])

,user_id,latitude,longitude,timestamp,visits,Авто,Дача,Спорт
0,11,59.572795,30.198666,1260759144,2.0,0.0,0.0,2.0
1,22,59.339912,30.193396,1260759179,1.0,1.0,0.0,0.0
2,55,59.815322,30.231700,1260759182,1.0,0.0,1.0,0.0
3,11,59.836001,30.773373,1260759185,2.0,0.0,0.0,2.0
4,77,60.017146,30.223552,1260758816,1.0,NaN,NaN,NaN
5,99,60.003151,30.645564,1260759205,NaN,1.0,0.0,0.0


Вот такая таблица может дать нам информацию не только по количеству покупок(визитов) в разрезе категорий товаров, но и о точках продаж(по координатам) и по временному интервалу покупок. Кроме того, координаты позволят идентифицировать клиентов в привязке к определённой точке продаж, которая наверняка находится в одной из часто посещаемых ими локаций.
Вывод: координаты нужно добавлять с привязкой времени.